# How to use regioinvent with premise

Note that running this entire notebook will take you around 2 to 3 hours if you choose the highest cutoff option.

To be able to use regioinvent, you will need:
- to install the brightway2 Python library (brightway2 and NOT brightway2.5), easier is to get it through activity-browser: https://github.com/LCA-ActivityBrowser/activity-browser
- a brightway project within which there is an ecoinvent database with either the version 3.9/3.9.1/3.10/3.10 cutoff
- to download the trade database stored here: https://doi.org/10.5281/zenodo.11583814

In [1]:
import pandas as pd
import numpy as np
import sys
# change the path here to wherever you stored the Regiopremise Python package
sys.path.append(r'C:\Users\matth\PycharmProjects\Regiopremise\src')
import regioinvent

The initialization of the Regioinvent class requires 3 arguments:
- the name of the brightway2 project where you store ecoinvent and where regioinvent will be added
- the name you gave your ecoinvent database within your brightway2 project
- the version of the ecoinvent database ('3.9', '3.9.1', '3.10' or '3.10.1')

In [2]:
ei_version = '3.10.1'
year = '2050'

In [3]:
regio = regioinvent.Regiopremise(
    bw_project_name='premise training',
    premise_database_name='ei_cutoff_3.10_image_SSP2-RCP19_2050 2025-04-09',
    ecoinvent_database_name=f'ecoinvent-{ei_version}-cutoff',
    ecoinvent_version=ei_version,
)

In [4]:
regio.fix_iam_location_codes()

First step of regioinvent is to spatialize the original premise database. This entails two steps:
- Creating a new biosphere database for spatialized elementary flows (e.g., Ammonia, CA-QC)
- Spatializing the elementary flows used within the premise database, based on the location of the process itself

In [4]:
regio.spatialize_my_premise()

2025-07-15 13:35:21,972 - Regiopremise - INFO - biosphere3_spatialized_flows already exists in this project.
2025-07-15 13:35:21,974 - Regiopremise - INFO - Extracting premise to wurst...


Getting activity data


100%|██████████| 34848/34848 [00:00<00:00, 89914.38it/s] 


Adding exchange data to activities


100%|██████████| 1004312/1004312 [00:36<00:00, 27561.23it/s]


Filling out exchange data


100%|██████████| 34848/34848 [00:02<00:00, 13440.59it/s]
2025-07-15 13:36:11,839 - Regiopremise - INFO - Spatializing premise...
Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:18


Title: Writing activities to SQLite3 database:
  Started: 07/15/2025 13:37:15
  Finished: 07/15/2025 13:38:34
  Total time elapsed: 00:01:18
  CPU %: 87.20
  Memory %: 24.37


This creates two databases in your brightway2 project:
- "_biosphere3_spatialized_flows_", which contains all the newly created spatialized elementary flows
- "the-name-of-your-premise-database regionalized", which is a copy of the original premise database but using spatialized elementary flows. You thus end up with two ecoinvent versions: the original one and a spatialized one.

Because elementary flows are now spatialized, you will need a specific LCIA method to operate with any flow (spatialized or not). The following function imports such methods. Currently available: "IW v2.1", "EF v3.1", "ReCiPe 2016 v1.03 (H)". Can also import all of them in one go.

In [5]:
regio.import_fully_regionalized_impact_method(lcia_method='IW v2.1')

2025-07-15 13:47:54,473 - Regiopremise - INFO - Importing the fully regionalized version of IMPACT World+ v2.1 for ecoinvent3.10.


If you want to go further in the regionalization, i.e., to create new national production processes and to rely on trade data to create regionalized consumption markets of premise, you can run the next function. There are 3 arguments:
- _trade_database_path_ which is the path where you stored the trade database you downloaded from Zenodo: https://doi.org/10.5281/zenodo.11583814
- _regioinvent_database_name_ which is simply the name that the created database will take
- _cutoff_ which is the cut-off (between 0 and 1) beyond which countries will be aggregated as RoW. For more info on what cutoff does, see section "Selection of countries for regionalization" of the Methodology.md file.

In [ ]:
regio.regionalize_premise_with_trade(
    trade_database_path=r'C:\Users\matth\PycharmProjects\Regiopremise\trade_data.db',
    regioinvent_database_name='regioinvent',
    cutoff=0.75,
)

Getting activity data


100%|██████████| 34848/34848 [00:00<00:00, 128265.01it/s]


Adding exchange data to activities


100%|██████████| 1004312/1004312 [00:49<00:00, 20291.14it/s]


Filling out exchange data


100%|██████████| 34848/34848 [00:08<00:00, 4299.02it/s]
2025-07-15 17:02:24,643 - Regiopremise - INFO - Extracting and formatting trade data...
2025-07-15 17:02:44,772 - Regiopremise - INFO - Regionalizing main inputs of internationally-traded products of premise...
 49%|████▉     | 967/1982 [02:28<01:39, 10.25it/s]

This automatically wrote the regioinvent database in your brightway project. So you can now go on brightway2 or AB to perform your LCAs normally with regioinvent.

## Debugging